# basic-002
https://chatgpt.com/c/680282ad-266c-800a-99bd-7eeb6918b9f9

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# ==== CONFIG ====
SEQ_LEN = 60
FUTURE_CANDLES = 100
COLUMNS = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL']

# ==== LOAD DATA ====
df_train = pd.read_csv("10_year_data.csv", sep=r'\s+', engine='python', names=["DATE", "TIME", "OPEN", "HIGH", "LOW", "CLOSE", "TICKVOL", "VOL", "SPREAD"], skiprows=1)
df_test = pd.read_csv("last_30_rows.csv", sep=r'\s+', engine='python', names=["DATE", "TIME", "OPEN", "HIGH", "LOW", "CLOSE", "TICKVOL", "VOL", "SPREAD"], skiprows=1)

# Drop unnecessary columns
df_train = df_train[COLUMNS]
df_test = df_test[COLUMNS]

# ==== SCALE ====
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(df_train)
test_scaled = scaler.transform(df_test)

# ==== PREPARE TRAINING SEQUENCES ====
X_train, y_train = [], []
for i in range(SEQ_LEN, len(train_scaled)):
    X_train.append(train_scaled[i-SEQ_LEN:i])
    y_train.append(train_scaled[i, :4])  # Predict OHLC
X_train, y_train = np.array(X_train), np.array(y_train)

# ==== MODEL ====
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(SEQ_LEN, len(COLUMNS))),
    LSTM(64),
    Dense(64, activation='relu'),
    Dense(4)  # Output: OHLC
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10, batch_size=64)

# ==== PREDICT NEXT 100 CANDLES ====
input_seq = test_scaled[-SEQ_LEN:].tolist()
predictions_scaled = []

for _ in range(FUTURE_CANDLES):
    x_input = np.array(input_seq[-SEQ_LEN:]).reshape(1, SEQ_LEN, len(COLUMNS))
    pred = model.predict(x_input, verbose=0)[0]
    next_candle = np.concatenate([pred, [input_seq[-1][-1]]])  # Use same TICKVOL as last known
    input_seq.append(next_candle)
    predictions_scaled.append(pred)

# ==== INVERSE SCALE ====
# Combine 30 real + 100 predicted candles
real_scaled = test_scaled[-30:, :4]
combined_scaled = np.vstack([real_scaled, predictions_scaled])

# Pad to full shape before inverse_transform
pad_tickvol = np.zeros((130, 1))
padded_scaled = np.hstack([combined_scaled, pad_tickvol])
real_plus_pred = scaler.inverse_transform(padded_scaled)[:, :4]  # Get OHLC

# ==== PLOT ====
plt.figure(figsize=(14, 6))
plt.plot(range(30), real_plus_pred[:30, 3], label='Real CLOSE')  # CLOSE is index 3
plt.plot(range(30, 130), real_plus_pred[30:, 3], label='Predicted CLOSE')
plt.axvline(x=29, color='gray', linestyle='--')
plt.title('30 Real + 100 Predicted Future Candles (CLOSE Price)')
plt.xlabel('Candle Index')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()
